## How to run your customized workflow?

### Requirement
You need to have an OpenAI API Key.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API Key"

### Set overall task & Customize your own workflow
Firstly, you need to define your overall task. After that, you can freely define your own workflow.  
Following attributes need to be provided for each subtask:
- **`id`**: The identifier for this subtask.  
  Format: `"taskx"`, where `x` is an Arabic numeral (e.g., `task1`, `task2`).

- **`objective`**: The specific content or goal of this subtask.

- **`agent_id`**: The identifier for the agent responsible for executing this subtask.

- **`next`**: A list of tasks that depend on the completion of this subtask.  
  Values in this list must be `id`s of previously defined tasks.

- **`prev`**: A list of tasks that this subtask depends on.  
  Values in this list must be `id`s of previously defined tasks.

- **`agent`**: The name of the agent responsible for completing this subtask.  
  The same `agent_id` must always correspond to the same `agent`.

Note That：
- Your workflow **must not** contain any form of circular dependencies.
- Ensure that all `next` and `prev` values reference valid `id`s of previously defined tasks.
- The `agent_id` and `agent` must remain consistent across all tasks.

In [ ]:
from workflow import Task, Workflow

# Customize your overall task here.
overall_task = '''Develop a Rock-Paper-Scissors game with UI in python. 
    Player will play against a naive AI player. 
    Please show the choices of players and the result of game.'''

# Use a list of dictionaries to initialize task data here.
tasks_data = [
    {
        "id": "task0",
        "objective": "Design and implement the game's user interface using tkinter, including buttons, computer's choice and result display in python.",
        "agent_id": 0,
        "next": ["task2"],
        "prev": [],
        "agent": "UI Designer",
    },
    {
        "id": "task1",
        "objective": "Design a naive AI and core game logic to determine the winner based on player and AI choices in python.",
        "agent_id": 1,
        "next": ["task2"],
        "prev": [],
        "agent": "Game Logic Developer",
    },
    {
        "id": "task2",
        "objective": "Combine the UI and game logic, test the game, and prepare it for deployment.",
        "agent_id": 2,
        "next": [],
        "prev": ["task0", "task1"],
        "agent": "Integration Specialist",
    },
]

# Create the workflow
workflow = Workflow(tasks={task_data["id"]: Task(**task_data) for task_data in tasks_data})

### Run your workflow

In [ ]:
import asyncio
import nest_asyncio
import json
from flow import Flow
from summary import Summary



manager = Flow(overall_task=overall_task, enable_refine=False, refine_threhold=0, n_candidate_graphs=0, workflow=workflow)
nest_asyncio.apply()
asyncio.run(manager.run_async())


workflow_data = {
    tid: task.__dict__ for tid, task in manager.workflow.tasks.items()
}
with open('manually_result.json', 'w', encoding='utf-8') as file:
    json.dump(workflow_data, file, indent=4)

summary = Summary()

# Generate and save a summary of the workflow results
chat_result = summary.summary(overall_task, workflow_data)
with open("example.txt", "w", encoding="utf-8") as file:
    file.write(chat_result)

2025-02-16 22:49:30 [INFO] Executing task 'Design and implement the game's user interface using tkinter, including buttons computer's choice and result display in python.' with context: No completed previous tasks context available....
2025-02-16 22:49:30 [INFO] Task 'Design and implement the game's user interface using tkinter, including buttons computer's choice and result display in python.' started by agent '0'.
2025-02-16 22:49:30 [INFO] Executing task 'Write the core game logic to determine the winner based on player and computer choices in python.' with context: No completed previous tasks context available....
2025-02-16 22:49:30 [INFO] Task 'Write the core game logic to determine the winner based on player and computer choices in python.' started by agent '1'.


object: Design and implement the game's user interface using tkinter, including buttons computer's choice and result display in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.
object: Write the core game logic to determine the winner based on player and computer choices in python.
next: Task task2:
  Objective: Combine the UI and game logic, test the game, and prepare it for deployment.


2025-02-16 22:49:34 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-16 22:49:34 [INFO] Task task1 completed with result: {
  "game_logic": {
    "choices": ["rock", "paper", "scissors"],
    "determine_winner": {
      "player_choice": "string",
      "computer_choice": "string",
      "result": {
        "player_wins": "boolean",
        "computer_wins": "boolean",
        "tie": "boolean"
      }
    },
    "logic_function": "def determine_winner(player_choice, computer_choice):\n    if player_choice not in choices or computer_choice not in choices:\n        raise ValueError('Invalid choice')\n    result = {'player_wins': False, 'computer_wins': False, 'tie': False}\n    if player_choice == computer_choice:\n        result['tie'] = True\n    elif (player_choice == 'rock' and computer_choice == 'scissors') or \n         (player_choice == 'paper' and computer_choice == 'rock') or \n         (player_choice == 'scissors' and computer_choice 

object: Combine the UI and game logic, test the game, and prepare it for deployment.
next: No downstream objectives available.


2025-02-16 22:50:06 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-16 22:50:06 [INFO] Task task2 completed with result: {
  "combined_game": {
    "ui_design": {
      "framework": "tkinter",
      "components": {
        "window": {
          "title": "Rock-Paper-Scissors Game",
          "size": "400x300"
        },
        "labels": {
          "user_choice_label": {
            "text": "Your Choice:",
            "position": {
              "x": 50,
              "y": 50
            }
          },
          "computer_choice_label": {
            "text": "Computer's Choice:",
            "position": {
              "x": 50,
              "y": 100
            }
          },
          "result_label": {
            "text": "Result:",
            "position": {
              "x": 50,
              "y": 150
            }
          }
        },
        "buttons": {
          "rock_button": {
            "text": "Rock",
            "position":